# Geostatistics and Machine Learning

See [Chege](https://medium.com/ai-advances/geostatistics-meets-machine-learning-a-dynamic-duo-4bf03d340d1e)

**Requires**: an input dataset `spatial_data.csv` containing geospatila data.

**Interest**: clear step-by-step method for Geostat + ML approach...

In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from pykrige.ok import OrdinaryKriging
import matplotlib.pyplot as plt

# Example Data: Let's assume you have spatial data in a CSV file
# with columns: 'longitude', 'latitude', 'value'
data = pd.read_csv('spatial_data.csv')

# Step 1: Prepare the data
X = data[['longitude', 'latitude']].values  # Spatial coordinates
y = data['value'].values  # Target variable

# Step 2: Fit a variogram model using Ordinary Kriging
OK = OrdinaryKriging(
    data['longitude'], data['latitude'], data['value'],
    variogram_model='gaussian',
    verbose=False, enable_plotting=False
)

# Step 3: Create a grid for predictions (for visualization or further analysis)
grid_lon = np.linspace(data['longitude'].min(), data['longitude'].max(), 100)
grid_lat = np.linspace(data['latitude'].min(), data['latitude'].max(), 100)
grid_lon, grid_lat = np.meshgrid(grid_lon, grid_lat)
z, ss = OK.execute('grid', grid_lon, grid_lat)

# Optional: Visualize the kriging result (interpolated surface)
plt.imshow(z, origin='lower')
plt.title('Kriging Interpolated Surface')
plt.colorbar(label='Interpolated Value')
plt.show()

# Step 4: Machine Learning Model (Random Forest)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the Random Forest model
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Predict and evaluate
y_pred = rf.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"Random Forest MSE: {mse}")

# Step 5: Combine variogram features into the model (Advanced)
# For simplicity, this example skips this step but you can
# create custom features based on variogram results and include them in your ML model

# Step 6: Spatial Prediction
# Use the trained Random Forest model to predict across a spatial grid
spatial_predictions = rf.predict(np.c_[grid_lon.ravel(), grid_lat.ravel()])
spatial_predictions = spatial_predictions.reshape(grid_lon.shape)

# Optional: Visualize the Random Forest spatial predictions
plt.imshow(spatial_predictions, origin='lower', extent=(grid_lon.min(), grid_lon.max(), grid_lat.min(), grid_lat.max()))
plt.title('Random Forest Spatial Predictions')
plt.colorbar(label='Predicted Value')
plt.show()